In [1]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from IPython.display import Image, display

import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as tfl

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation
from keras.layers import BatchNormalization
import cv2
from PIL import Image

In [2]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
directory = "C:\\Users\\AatishNehe\\Downloads\\Helipad\\helipad 0 & helipad 1"

train_dataset = image_dataset_from_directory(directory,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='training',
                                             seed=42)
validation_dataset = image_dataset_from_directory(directory,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='validation',
                                             seed=42)

Found 14324 files belonging to 2 classes.
Using 11460 files for training.
Found 14324 files belonging to 2 classes.
Using 2864 files for validation.


In [3]:
class_names = train_dataset.class_names
print(class_names)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)

['Helipad', 'No_helipad']


In [4]:
def data_augmenter():

    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(RandomFlip('horizontal'))
    data_augmentation.add(RandomRotation(0.2))

    return data_augmentation

In [5]:
data_augmentation = data_augmenter()

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

image_batch, label_batch = next(iter(train_dataset))
print(label_batch)
image_var = tf.Variable(preprocess_input(image_batch))


tf.Tensor([1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 0 0 1], shape=(32,), dtype=int32)


In [6]:
def helipad_model(image_shape=IMG_SIZE, data_augmentation=data_augmenter()):
    input_shape = image_shape + (3,)
    base_model = tf.keras.applications.Xception(input_shape=input_shape,
                                                   include_top=False,  # <== Important!!!!
                                                   weights='imagenet')  # From imageNet

    for layer in base_model.layers[:]:
        layer.trainable = False or isinstance(layer, BatchNormalization)

    # base_model.trainable = False

    inputs = tf.keras.Input(shape=input_shape)

    x = data_augmentation(inputs)

    x = preprocess_input(x)

    x = base_model(x, training=False)

    x = tfl.GlobalAveragePooling2D()(x)

    x = tfl.Dropout(0.2)(x)

    outputs = tfl.Dense(1)(x)

    model = tf.keras.Model(inputs, outputs)

    return model


In [7]:
model2 = helipad_model(IMG_SIZE, data_augmentation)

base_learning_rate = 0.001

model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
model2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0     

In [8]:
base_model = model2.layers[4]
base_model.trainable = True
print("Number of layers in the base model: ", len(base_model.layers))
fine_tune_at = 100


for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

loss_function = tf.keras.losses.BinaryCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1 * base_learning_rate)
metrics = ['accuracy']

model2.compile(loss=loss_function, optimizer=optimizer, metrics=metrics)

Number of layers in the base model:  132


In [9]:
initial_epochs = 5
history = model2.fit(train_dataset, validation_data=validation_dataset, epochs=initial_epochs)

Epoch 1/5
 92/359 [======>.......................] - ETA: 18:00 - loss: 0.2516 - accuracy: 0.8815

KeyboardInterrupt: 

In [78]:
img_path = 'C:\\Users\\AatishNehe\\Downloads\\Helipad\\helipad 0 & helipad 1\\1\\CAH_0003.png'
image = Image.open(img_path)
image = image.convert('RGB')
image = image.resize((224,224))
numpydata = np.asarray(image)

print(image.mode)
print(numpydata.shape)

RGB
(224, 224, 3)


In [83]:
for image_batch, labels_batch in validation_dataset:
    
    y_pred = model2.predict(image_batch)

1/1 [==============================] - 2s 2s/step


In [85]:
print(y_pred)

[[ 0.10848328]
 [-0.31018853]
 [-0.36221057]
 [-0.05268236]
 [ 0.06560268]
 [-0.23050228]
 [-0.2641843 ]
 [ 0.08776212]
 [-0.27487972]
 [-0.81078947]
 [-0.07348615]
 [ 0.05363303]
 [-0.10994142]
 [-0.25294968]
 [ 0.73570335]
 [-0.31423458]]
